In [1]:
# import libraries
import os
import numpy as np
import pandas as pd
import cv2
import gc
from tqdm import tqdm
from glob import glob

#### Step-1 and -2
- Collect all data
- labelling

In [2]:
dirs = os.listdir('data')
images_path = []
labels = []
for folder in dirs:
    path = glob('./data/{}/*.jpg'.format(folder))
    label = ['{}'.format(folder)]*len(path)
    ## append
    images_path.extend(path)
    labels.extend(label)

In [3]:
images_path

['./data/Mask\\00000.jpg',
 './data/Mask\\00001.jpg',
 './data/Mask\\00002.jpg',
 './data/Mask\\00003.jpg',
 './data/Mask\\00004.jpg',
 './data/Mask\\00005.jpg',
 './data/Mask\\00006.jpg',
 './data/Mask\\00007.jpg',
 './data/Mask\\00008.jpg',
 './data/Mask\\00009.jpg',
 './data/Mask\\00010.jpg',
 './data/Mask\\00011.jpg',
 './data/Mask\\00012.jpg',
 './data/Mask\\00013.jpg',
 './data/Mask\\00014.jpg',
 './data/Mask\\00015.jpg',
 './data/Mask\\00016.jpg',
 './data/Mask\\00017.jpg',
 './data/Mask\\00018.jpg',
 './data/Mask\\00019.jpg',
 './data/Mask\\00020.jpg',
 './data/Mask\\00021.jpg',
 './data/Mask\\00022.jpg',
 './data/Mask\\00023.jpg',
 './data/Mask\\00024.jpg',
 './data/Mask\\00025.jpg',
 './data/Mask\\00026.jpg',
 './data/Mask\\00027.jpg',
 './data/Mask\\00028.jpg',
 './data/Mask\\00029.jpg',
 './data/Mask\\00030.jpg',
 './data/Mask\\00031.jpg',
 './data/Mask\\00032.jpg',
 './data/Mask\\00033.jpg',
 './data/Mask\\00034.jpg',
 './data/Mask\\00035.jpg',
 './data/Mask\\00036.jpg',
 

#### Step-3
- Face Detection

In [4]:
img_path = images_path[1]
img = cv2.imread(img_path)

In [5]:
cv2.imshow('original', img)
cv2.waitKey()
cv2.destroyAllWindows()

In [6]:
# face detection
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt', 
                                               './models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [7]:
def face_detection_dnn(img):
    # blob from image (rgb mean subtraction image) 
    image = img.copy()
    h,w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1, (300,300), (104,117,123), swapRB=True)
    # get the detection
    face_detection_model.setInput(blob)
    detections = face_detection_model.forward()
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2] # confidence score
        if confidence > 0.5:
            box = detections[0, 0, i, 3:7]*np.array([w,h,w,h])
            box = box.astype(int)
            print(box)
            pt1 = (box[0], box[1])
            pt2 = (box[2], box[3])
            #cv2.rectangle(image, pt1, pt2, (0,255,0), 2)
            roi = image[box[1]:box[3], box[0]:box[2]]

            return roi
    return None


In [8]:
img_roi = face_detection_dnn(img)

[216 142 802 801]


In [9]:
cv2.imshow('roi', img_roi)
cv2.imshow('original', img)
cv2.waitKey()
cv2.destroyAllWindows()

#### Step 5
- Blob from image

In [15]:
def datapreprocess(img):
    # blob from image (rgb mean subtraction image)
    face = face_detection_dnn(img)
    if face is not None:
        # computing blob from image
        blob = cv2.dnn.blobFromImage(face, 1, (100,100), (104,117,123), swapRB=True)
        blob_squeeze = np.squeeze(blob).T
        blob_rotate = cv2.rotate(blob_squeeze, cv2.ROTATE_90_CLOCKWISE)
        blob_flip = cv2.flip(blob_rotate, 1)
        # remove negative values and normalize
        img_norm = np.maximum(blob_flip, 0)/blob_flip.max()

        return img_norm
    else:
        return None


#### Apply to all images and Append in a List

In [18]:
#len(images_path)

data_img = []
label_img = []
i = 0
for path, label in tqdm(zip(images_path, labels), desc='preprocessing'):
    img = cv2.imread(path)
    process_img = datapreprocess(img)
    if process_img is not None:
        data_img.append(process_img)
        label_img.append(label)
        
    i += 1
    if i%100 == 0:
        gc.collect()

preprocessing: 3it [00:00, 26.30it/s]

[110 139 757 825]
[216 142 802 801]
[203 168 771 860]
[205 132 836 830]
[273 189 805 825]


preprocessing: 9it [00:00, 24.07it/s]

[252 182 758 859]
[314 157 930 849]
[126 175 788 804]
[184 117 785 818]
[ 51 127 771 875]


preprocessing: 15it [00:00, 24.80it/s]

...

[233 196 822 945]
[260 192 873 930]
[163 190 756 913]


preprocessing: 9998it [08:11, 15.37it/s]

[210 239 768 832]
[ 267  103 1008  887]
[302 201 853 931]
[274 102 983 877]


preprocessing: 10000it [08:12, 20.32it/s]

[233 204 775 948]


In [22]:
X = np.array(data_img)
y = np.array(label_img)

In [23]:
X.shape, y.shape

((9959, 100, 100, 3), (9959,))

In [24]:
np.savez('./data/data_preprocess.npz', X, y)